In [ ]:
import pyarrow as pa
import pyarrow.parquet as pq
import pandas as pd
from collections import defaultdict
import numpy as np
import random
import gc
import csv
import time
from gensim.models import Word2Vec
import pickle
import sys
import os
from copy import deepcopy
import logging
from collections import Counter
import sqlite3
from tqdm import tqdm
import networkx as nx


In [ ]:
SAVE_DIR = '/home/ps2575/proj/wikibench_v.1.1_data_backup/00.data/00.wikidata/02.wikidata_embedding/'

In [ ]:
input_file_path = '/home/ps2575/proj/wikibench_v.1.1_data_backup/00.data/00.wikidata/01.wikidata_merged/01.wikidata_merged/triples_final_final.parquet'
triples_df = pq.read_table(input_file_path).to_pandas()

In [ ]:
df = triples_df

In [ ]:
len(triples_df)

In [ ]:
from tqdm import tqdm
import networkx as nx

G = nx.Graph()

# 데이터프레임의 행을 반복하면서 그래프에 엣지 추가
for idx, row in tqdm(df.iterrows(), total=df.shape[0]):
    G.add_edge(row['object'], row['subject'])

In [ ]:
# 연결 요소를 찾아 리스트로 변환합니다.
connected_components = list(nx.connected_components(G))

# 각 연결 요소의 크기를 계산합니다.
component_sizes = [len(c) for c in connected_components]

# 가장 큰 연결 요소를 찾습니다.
largest_component = max(connected_components, key=len)

# 가장 큰 연결 요소만을 포함하는 그래프를 생성합니다.
G = G.subgraph(largest_component).copy()

In [ ]:
from collections import defaultdict
import numpy as np
import random
import gc
import csv
import time
from gensim.models import Word2Vec
import pickle
import sys
import os
from copy import deepcopy
import logging
from collections import Counter
import sqlite3


class RandomWalkIterator():
    import numpy as np
    def __init__(self, generator_function, iter_dictionary, max_iteration=1, max_walking_dist=20, random_seed=0):
        self.generator_function = generator_function
        self.iter_dictionary = iter_dictionary
        self.max_iteration = max_iteration
        self.max_walking_dist = max_walking_dist
        self.random_seed = random_seed       
        self.generator = self.generator_function(random_seed, iter_dictionary, max_iteration, max_walking_dist)

    def __iter__(self):
        self.generator = self.generator_function(self.random_seed, self.iter_dictionary, 
                                                 self.max_iteration, self.max_walking_dist)
        return self

    def __next__(self):
        result = next(self.generator)
        if result is None:
            raise StopIteration
        else:
            return result
    
    def __len__(self): 
        return self.max_iteration * len(self.iter_dictionary)
        
def RandomWalkGenerator(random_seed, iter_dictionary, max_iteration, max_walking_dist):
    #print(type(iter_dictionary))
    random.seed(random_seed)
    np.random.seed(random_seed)
    nodes = sorted(list(iter_dictionary.keys()))
    random.shuffle(nodes)
    for source in nodes:
        for it in range(max_iteration):
            rw_seq = []
            current_node = source
            rw_seq.append(str(source))
            for st in range(max_walking_dist):
                next_node = random.choice(iter_dictionary[current_node])
                rw_seq.append(str(next_node))
                current_node = next_node
            yield rw_seq

def RandomWalkGeneratorDirected(random_seed, iter_dictionary, max_iteration, max_walking_dist):
    #print(type(iter_dictionary))
    import numpy as np
    
    random.seed(random_seed)
    np.random.seed(random_seed)
    nodes = sorted(list(iter_dictionary.keys()))
    random.shuffle(nodes)
    
    for source in nodes:
        for it in range(max_iteration):
            rw_seq = []
            current_node = source
            rw_seq.append(str(source))
            
            for st in range(max_walking_dist):
                if(len(iter_dictionary[current_node])==0):
                    break
                    
                next_node = random.choice(iter_dictionary[current_node])
                rw_seq.append(str(next_node))
                current_node = next_node
                
            yield rw_seq


In [ ]:
neighbor_dict = nx.to_dict_of_lists(G)
num_walkers = 10
seq_length = 10
randseed = 0
vector_size = 64
word_window = 5
basemodel_it = RandomWalkIterator(RandomWalkGenerator, neighbor_dict, num_walkers, seq_length, randseed)
base_model = Word2Vec(sentences=basemodel_it, vector_size=vector_size, window=word_window, min_count=1, workers=32, sg=1)

In [ ]:
base_model.wv.save(f'{SAVE_DIR}node_vectors.kv')

In [ ]:
node_vectors.similarity("Q8539","Q149481")


In [ ]:
node_vectors.similarity("Q8539","Q790090")
